# This file can be used as example for using Reptile meta-learning algorithm for EEG-data
## Data loading from BRAIN DECODE

In [ ]:
from braindecode.datasets.moabb import MOABBDataset
import os
from braindecode.datautil.preprocess import (exponential_moving_standardize, preprocess, Preprocessor)
from braindecode.datautil.windowers import create_windows_from_events
import pandas as pd
subject_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9] #full list of subjects
ds = MOABBDataset(dataset_name="BNCI2014001", subject_ids=subject_ids) #load BCI IV data from brain decode
low_cut = 4. #low cut freq
high_cut = 38. #hi cut freq
factor_new = 1e-3
init_block_size = 1000
preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),
    Preprocessor(lambda x: x*1e6),
    Preprocessor('filter', l_freq=low_cut, h_freq=high_cut),
    Preprocessor(exponential_moving_standardize, factor_new=factor_new, init_block_size=init_block_size)
]
preprocess(ds, preprocessors)
trial_start_ofset = -0.5
sfreq = ds.datasets[0].raw.info['sfreq']
assert all([d.raw.info['sfreq'] == sfreq for d in ds.datasets])
trial_start_ofset_samples = int(trial_start_ofset * sfreq)
w_ds = create_windows_from_events(
    ds,
    trial_start_offset_samples=trial_start_ofset_samples,
    trial_stop_offset_samples=0,
    preload=True
)
X = []
Y = []
Window = []
dsub = w_ds.split("subject")
os.mkdir('data')
for sub in subject_ids:
    sub_path = "./data/subj_" + str(sub)
    os.mkdir(sub_path)
    sds = dsub[str(sub)].split("session")
    for x, y, window in sds["session_T"]:
        X.append(x)
        Y.append(y)
        Window.append(window)
    data_dict = {'X': X, 'Y': Y, 'window_data': Window}
    df = pd.DataFrame.from_dict(data_dict)
    path = sub_path + '/' + 'train_data.csv'
    df.to_csv(path)
    X.clear()
    Y.clear()
    Window.clear()
    for x, y, window in sds["session_E"]:
        X.append(x)
        Y.append(y)
        Window.append(window)
    data_dict = {'X': X, 'Y': Y, 'window_data': Window}
    df = pd.DataFrame.from_dict(data_dict)
    path = sub_path + '/' + 'validation_data.csv'
    df.to_csv(path)